In [1]:
import gradio as gr
import os
import tempfile
import numpy as np
import soundfile as sf
from dotenv import load_dotenv
from openai import OpenAI
import random

from setup import model

load_dotenv()
client = OpenAI()


ModuleNotFoundError: No module named 'setup'

In [ ]:
def translate_word(word, source_lang="da", target_lang="en"):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": f"""You are a translator from {source_lang} to {target_lang}. 
             Your output should be similar to a dictionary definition.
             Include plural cases, verb forms, genders, cases and other relevant information."""},
            {"role": "user", "content": f"Translate the word: {word}"}
        ]
    )
    return response.choices[0].message.content.strip()

translate_word(word, source_lang="da", target_lang="en")

In [ ]:
def generate_sentence(word, language="en"):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": f"You are a sentence generator in {language}."},
            {"role": "user", "content": f"Generate a simple sentence using the word: {word}"}
        ]
    )
    return response.choices[0].message.content.strip()

In [12]:
def dictionarize_word(client, word, source_lang="da", english_translation = None):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": f"""You are a dictionary assistant in lanuage {source_lang}. 
             You will get a word and you have to provide the primitive form of given word and make it look like the way it is in a dictionary.
             Make sure to start by listing the words primitive form!
             Include plural cases, verb forms, genders, cases and other relevant information.
             Use the following template:
             ```
             {{Word}} (Part of Speech)
                Meaning in English: {{English translation}}
                Bøjning: {{Inflections}}
                Udtale: {{Pronunciation}}
                    Additional forms/pronunciation (if applicable): {{Additional Pronunciation}}
                    Oprindelse: {{Origin/etymology}}
            ```
            Examples using the template:
            ``` 
            være (verbum)
            Meaning in English: to be
            Bøjning: er, var, -t (talesprogsefterlignende også: vær')
            Udtale: [ˈvεːʌ]

                præsens: [ˈæɐ̯]
                præteritum: [ˈvɑ]
                præteritum participium: [ˈvεːʌð]
                vær', i la' vær': [ˈvεɐ̯ˀ]
                Oprindelse: norrønt vera, vesa, oldengelsk wesan, ikke beslægtet med præsens er, som er indoeuropæisk: norrønt es, er, engelsk is, latin est
            ```
            ```
            hund (substantiv, fælleskøn)

            Åbn overblik
            Vis overblik
            Meaning in English: dog
            Bøjning: -en, -e, -ene
            Udtale: [ˈhunˀ]

                i sammensætning hunde-: [ˈhunə-]
                Oprindelse: norrønt hundr, tysk Hund, beslægtet med græsk kynos, af uvis oprindelse
            ```
             """},
            {"role": "user", "content": f"Transform the word: {word}. The user wanted to use the following English translation: {english_translation or 'No translation provided.'}"}
        ]
    )
    return response.choices[0].message.content.strip()

output = dictionarize_word(client, "hunden", source_lang="da", english_translation="dog")
output = dictionarize_word(client, "hunden", source_lang="da")

print(output)

hund (substantiv, fælleskøn)
    Meaning in English: dog
    Bøjning: hund, hunde, hundene
    Udtale: [ˈhunˀ]

        i sammensætning hunde-: [ˈhunə-]
        Oprindelse: norrønt hundr, tysk Hund, beslægtet med græsk kynos, af uvis oprindelse


In [5]:
def html_fixer(html):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": f"You are an assistant turning HTML into easy to read text."},
            {"role": "user", "content": f"Turn this into an easy to consume text. This is a dictionary definition. {html}"}
        ]
    )
    return response.choices[0].message.content.strip()

div = """
<div class="artikel">
<div class="definitionBoxTop"><span class="match">hund<span class="super">1</span></span> <span class="tekstmedium allow-glossing">substantiv, fælleskøn</span></div>
<div>
<div class="overblikBox" id="id-overblik">
<div id="minus-overblik" style="display: none;">
<div class="openClose">
<a accesskey="l" href="javascript:void(0)" onclick="lunchboxClose('overblik');" title="Luk overblikket (l)">
<img alt="Luk Overblik" height="16" src="minus.gif" width="16"/>
</a>
</div>
<div class="overblikHeader">
<a onclick="lunchboxClose('overblik');">Skjul overblik</a>
</div>
</div>
<div id="plus-overblik">
<div class="openClose">
<a accesskey="å" href="javascript:void(0)" onclick="lunchboxOpen('overblik');" title="Åbn overblikket (å)">
<img alt="Åbn overblik" height="16" src="plus.gif" width="16"/>
</a>
</div>
<div class="overblikHeader">
<a onclick="lunchboxOpen('overblik');">Vis overblik</a>
</div>
</div>
<div class="overblikBody" id="content-overblik" style="display:none;">
<div class="overblikItem">
<div class="openClose">
<img alt="Luk overblik betydninger" height="16" id="minus-overblik-betyd" onclick="lunchboxClose('overblik-betyd');" src="minus.gif" width="16"/>
<img alt="Åbn overblik betydninger" height="16" id="plus-overblik-betyd" onclick="lunchboxOpen('overblik-betyd');" src="plus.gif" style="display: none;" width="16"/>
</div>
<div class="overblikItemHeader">
<a accesskey="b" href="https://ordnet.dk/ddo/ordbog?query=hund#betydninger" title="Spring til Betydninger (b)">Betydninger</a>
</div>
<div class="overblikItemBody" id="content-overblik-betyd">
<ol class="overblikList">
<li>
<a href="https://ordnet.dk/ddo/ordbog?query=hund#betydning-1" onclick="mark('betydning-1');">pattedyr som har god lugtesans og hørelse, og som holdes som kæledyr og bruges til jagt m.m.</a>
<ol>
<li>
<a href="https://ordnet.dk/ddo/ordbog?query=hund#betydning-1-1" onclick="mark('betydning-1-1');"><span class="stempelNoBorder">ZOOLOGI</span> rovdyr som tilhører hundefamilien, ofte med spids snude og lang hale</a>
</li>
</ol>
</li>
<li>
<a href="https://ordnet.dk/ddo/ordbog?query=hund#betydning-2" onclick="mark('betydning-2');">foragtelig person</a>
</li>
<li>
<a href="https://ordnet.dk/ddo/ordbog?query=hund#betydning-3" onclick="mark('betydning-3');"><span class="stempelNoBorder">jargon</span> solid firkantet plade der er påmonteret hjul, og som bl.a. bruges til at flytte tunge ting på</a>
</li>
</ol>
</div>
</div>
<div class="overblikItem">
<div class="openClose">
<img alt="vis faste udtryk" height="16" id="plus-overblik-udtryk" onclick="lunchboxOpen('overblik-udtryk');" src="plus.gif" style="display: none;" width="16"/>
<img alt="Luk overblik for faste udtryk" height="16" id="minus-overblik-udtryk" onclick="lunchboxClose('overblik-udtryk');" src="minus.gif" width="16"/>
</div>
<div class="overblikItemHeader">
<a accesskey="u" href="https://ordnet.dk/ddo/ordbog?query=hund#faste-udtryk" title="Spring til faste udtryk (u)">Faste udtryk</a>
</div>
<div class="overblikItemBody" id="content-overblik-udtryk">
<ol class="overblikList">
<li>
<a href="https://ordnet.dk/ddo/ordbog?query=hund#udtryk-1" onclick="mark('udtryk-1');">
<span>dér ligger hunden begravet</span>
</a>
</li>
<li>
<a href="https://ordnet.dk/ddo/ordbog?query=hund#udtryk-2" onclick="mark('udtryk-2');">
<span>det er mig, sagde hunden</span>
</a>
</li>
<li>
<a href="https://ordnet.dk/ddo/ordbog?query=hund#udtryk-3" onclick="mark('udtryk-3');">
<span>en hund i et spil kegler</span>
</a>
</li>
<li>
<a href="https://ordnet.dk/ddo/ordbog?query=hund#udtryk-4" onclick="mark('udtryk-4');">
<span>flyvende hund</span>
</a>
</li>
<li>
<a href="https://ordnet.dk/ddo/ordbog?query=hund#udtryk-5" onclick="mark('udtryk-5');">
<span>gå i hundene</span>
</a>
</li>
<li>
<a href="https://ordnet.dk/ddo/ordbog?query=hund#udtryk-6" onclick="mark('udtryk-6');">
<span>hård hund</span>
</a>
</li>
<li>
<a href="https://ordnet.dk/ddo/ordbog?query=hund#udtryk-7" onclick="mark('udtryk-7');">
<span>leve som hund og kat</span>
</a>
</li>
<li>
<a href="https://ordnet.dk/ddo/ordbog?query=hund#udtryk-8" onclick="mark('udtryk-8');">
<span>man kan ikke lære en gammel hund nye kunster</span>
</a>
</li>
<li>
<a href="https://ordnet.dk/ddo/ordbog?query=hund#udtryk-9" onclick="mark('udtryk-9');">
<span>man skal ikke skue hunden på hårene</span>
</a>
</li>
<li>
<a href="https://ordnet.dk/ddo/ordbog?query=hund#udtryk-10" onclick="mark('udtryk-10');">
<span>røde hunde</span>
</a>
</li>
<li>
<a href="https://ordnet.dk/ddo/ordbog?query=hund#udtryk-11" onclick="mark('udtryk-11');">
<span>vittig hund</span>
</a>
</li>
<li>
<a href="https://ordnet.dk/ddo/ordbog?query=hund#udtryk-12" onclick="mark('udtryk-12');">
<span>være en hund efter noget</span>
</a>
</li>
</ol>
</div>
</div>
<div class="overblikItem">
<div class="openClose"> </div>
<div class="overblikItemHeader">
<a accesskey="o" href="https://ordnet.dk/ddo/ordbog?query=hund#orddannelser" title="Spring til orddannelser (o)">Orddannelser</a>
</div>
<div class="overblikItemBody"></div>
</div>
</div>
</div>
</div>
<div class="definitionBox" id="id-boj">
<span class="stempel">Bøjning</span>
<span class="tekstmedium allow-glossing">-en, -e, -ene</span>
</div>
<div class="definitionBox details" id="id-udt">
<span class="stempel">Udtale</span>
<span class="tekstmedium allow-glossing"><span class="lydskrift"><span class="diskret">[</span>ˈhunˀ<span class="diskret">]</span> <audio id="11021628_1" src=""><div class="hiddenStructure"><a href="https://static.ordnet.dk/mp3/11021/11021628_1.mp3" id="11021628_1_fallback"> </a></div></audio><img onclick="playSound('11021628_1'); _gaq.push(['_trackPageview', '/static/lyd/11021628_1']);" src="speaker.gif" style="cursor: pointer;"/></span> <span class="dividerDouble"> </span> <span class="diskret">i sammensætning <span class="ordform">hunde-</span>:</span> <span class="lydskrift"><span class="diskret">[</span>ˈhunə-<span class="diskret">]</span></span></span>
<span class="tipIkon">
<a href="../ddo/artiklernes-opbygning/udtale?set_language=da" title="hjælp til læsning af lydskriften">
<img alt="tip" height="18" src="tip_ikon_mini.gif" width="18"/>
</a>
</span>
</div>
<div class="definitionBox details" id="id-ety">
<span class="stempel">Oprindelse</span>
<span class="tekstmedium allow-glossing">norrønt <span class="ordform">hundr</span>, tysk <span class="ordform">Hund</span><span class="dividerDot"> </span>beslægtet med græsk <span class="ordform">kynos</span>, af uvis oprindelse</span>
</div>
<div class="openClose2">
<img alt="Skjul Betydning" height="16" id="minus-betydninger" onclick="lunchboxClose('betydninger');" src="minus.gif" width="16"/>
<img alt="Skjul Betydning" height="16" id="plus-betydninger" onclick="lunchboxOpen('betydninger');" src="plus.gif" style="display: none;" width="16"/>
</div>
<div class="definitionBox">
<h2 id="betydninger">Betydninger</h2>
</div>
<div id="content-betydninger">
<div class="definitionNumber">1.
        </div>
<div class="definitionIndent">
<div class="definitionBox" id="betydning-1">
<span class="tekstmedium"><span class="definition">pattedyr som har god lugtesans og hørelse, og som holdes som kæledyr og bruges til jagt m.m.</span><span class="dividerDot"> </span>tilhører hundefamilien</span>
</div>
<div class="definitionBox onym">
<span class="stempel">Synonym</span>
<span class="inlineList">Canis familiaris</span>
</div>
<div class="definitionBox rel-begreber" id="plus-ddb-betydning-1">
<span id="ddb_short_betydning-1">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="performddbiddoQuery('21033604', 'ddb_long_betydning-1_insert', 'ddb_short_betydning-1', 'ddb_long_betydning-1')">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList"><span class="level1 motherkey allow-glossing"><a href="?entry_id=11043693&amp;def_id=21068477&amp;query=hund">rovdyr</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing">menneskets bedste ven</span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?subentry_id=59011918&amp;def_id=21094439&amp;query=hund">firbenet ven</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11021858&amp;def_id=21033934&amp;query=hund">hvalp</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11021645&amp;def_id=21033639&amp;query=hund">hundehvalp</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=50010180&amp;def_id=28100135&amp;query=hund">tævehvalp</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=12005020&amp;def_id=21092465&amp;query=hund">unghund</a></span><a href="javascript:void(0)" title="Se flere oplysninger fra Den Danske Begrebsordbog"><span class="ddb-open" id="ddb_open_betydning-1" onclick="performddbiddoQuery('21033604', 'ddb_long_betydning-1_insert', 'ddb_short_betydning-1', 'ddb_long_betydning-1')" role="button">...vis mere</span></a></span>
</span>
<div class="ddb-details" id="ddb_long_betydning-1" style="display:none;">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="$('#ddb_long_betydning-1').hide();$('#ddb_short_betydning-1').show();">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span id="ddb_long_betydning-1_insert"> </span>
<a href="javascript:void(0)" title="Skjul oplysningerne fra Den Danske Begrebsordbog">
<span class="ddb-close" onclick="$('#ddb_long_betydning-1').hide();$('#ddb_short_betydning-1').show()" role="button">...vis mindre</span>
</a>
<div>
<span class="tipIkon">
<img alt="tip" height="18" src="tip_ikon_mini.gif" width="18"/>
</span>
<a href="https://ordnet.dk/ddb">Læs mere om Den Danske Begrebsordbog</a>
</div>
</span>
</div>
</div>
<div class="definitionBox details">
<span class="stempel">Eksempler</span>
<span class="inlineList">bidske hunde<span class="kIkon"><a href="https://ordnet.dk/korpusdk/qconc?query=bidske hunde" title='Slå "bidske hunde" op i KorpusDK'><img alt="ikon for korpussøgning" height="11" src="k-ikon.gif" width="11"/></a></span><span class="dividerSmall"> </span> løse hunde<span class="kIkon"><a href="https://ordnet.dk/korpusdk/qconc?query=løse hunde" title='Slå "løse hunde" op i KorpusDK'><img alt="ikon for korpussøgning" height="11" src="k-ikon.gif" width="11"/></a></span><span class="dividerDouble"> </span> lufte hunden<span class="kIkon"><a href="https://ordnet.dk/korpusdk/qconc?query=lufte hunden" title='Slå "lufte hunden" op i KorpusDK'><img alt="ikon for korpussøgning" height="11" src="k-ikon.gif" width="11"/></a></span><span class="dividerSmall"> </span> hunde i snor<span class="kIkon"><a href="https://ordnet.dk/korpusdk/qconc?query=hunde i snor" title='Slå "hunde i snor" op i KorpusDK'><img alt="ikon for korpussøgning" height="11" src="k-ikon.gif" width="11"/></a></span></span>
</div>
<div class="definitionBox">
<div class="citat-box"><span class="citat">En hund kommer løbende, gøende, springer op ad ham og han forsøger at skubbe den væk</span> <a class="kildepop" href="javascript:void(0);"><span class="kilde">PeAsmu92</span><span class="popup">Peter Asmussen: Om natten. Gyldendal, 1992.</span></a></div>
</div>
<div class="strokeGrey"> </div>
</div>
<div class="definitionIndent">
<div class="definitionNumber">1.a</div>
<div class="definitionIndent">
<div class="definitionBox" id="betydning-1-1">
<span class="tekstmedium"><span class="stempelNoBorder">ZOOLOGI</span> <span class="definition">rovdyr som tilhører hundefamilien, ofte med spids snude og lang hale<span class="dividerStroke"> </span>fx ulv, ræv eller sjakal</span></span>
</div>
<div class="definitionBox onym">
<span class="stempel">Synonym</span>
<span class="inlineList"><span class="diskret">familien</span> Canidae</span>
</div>
<div class="definitionBox rel-begreber" id="plus-ddb-betydning-1-1">
<span id="ddb_short_betydning-1-1">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="performddbiddoQuery('21033607', 'ddb_long_betydning-1-1_insert', 'ddb_short_betydning-1-1', 'ddb_long_betydning-1-1')">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList"><span class="level3 allow-glossing"><a href="?entry_id=11029202&amp;def_id=21045693&amp;query=hund">kødæder</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11035197&amp;def_id=21055189&amp;query=hund">mårhund</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11009124&amp;def_id=21013625&amp;query=hund">dingo</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11022165&amp;def_id=21034452&amp;query=hund">hyænehund</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11022164&amp;def_id=21034450&amp;query=hund">hyæne</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11046127&amp;def_id=21072524&amp;query=hund">sjakal</a></span><a href="javascript:void(0)" title="Se flere oplysninger fra Den Danske Begrebsordbog"><span class="ddb-open" id="ddb_open_betydning-1-1" onclick="performddbiddoQuery('21033607', 'ddb_long_betydning-1-1_insert', 'ddb_short_betydning-1-1', 'ddb_long_betydning-1-1')" role="button">...vis mere</span></a></span>
</span>
<div class="ddb-details" id="ddb_long_betydning-1-1" style="display:none;">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="$('#ddb_long_betydning-1-1').hide();$('#ddb_short_betydning-1-1').show();">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span id="ddb_long_betydning-1-1_insert"> </span>
<a href="javascript:void(0)" title="Skjul oplysningerne fra Den Danske Begrebsordbog">
<span class="ddb-close" onclick="$('#ddb_long_betydning-1-1').hide();$('#ddb_short_betydning-1-1').show()" role="button">...vis mindre</span>
</a>
<div>
<span class="tipIkon">
<img alt="tip" height="18" src="tip_ikon_mini.gif" width="18"/>
</span>
<a href="https://ordnet.dk/ddb">Læs mere om Den Danske Begrebsordbog</a>
</div>
</span>
</div>
</div>
<div class="strokeGrey"> </div>
</div>
</div>
<div class="definitionNumber">2.
        </div>
<div class="definitionIndent">
<div class="definitionBox" id="betydning-2">
<span class="tekstmedium">
<span class="definition">foragtelig person</span>
</span>
<span class="stempel">SPROGBRUG</span>
<span class="tekstnormal">nedsættende</span>
</div>
<div class="definitionBox rel-begreber" id="plus-ddb-betydning-2">
<span id="ddb_short_betydning-2">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="performddbiddoQuery('21033605', 'ddb_long_betydning-2_insert', 'ddb_short_betydning-2', 'ddb_long_betydning-2')">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span class="level2 allow-glossing">
<a href="?entry_id=12003444&amp;def_id=21089801&amp;query=hund">tåbe</a>
</span>
<a href="javascript:void(0)" title="Se flere oplysninger fra Den Danske Begrebsordbog">
<span class="ddb-open" id="ddb_open_betydning-2" onclick="performddbiddoQuery('21033605', 'ddb_long_betydning-2_insert', 'ddb_short_betydning-2', 'ddb_long_betydning-2')" role="button">...vis mere</span>
</a>
</span>
</span>
<div class="ddb-details" id="ddb_long_betydning-2" style="display:none;">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="$('#ddb_long_betydning-2').hide();$('#ddb_short_betydning-2').show();">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span id="ddb_long_betydning-2_insert"> </span>
<a href="javascript:void(0)" title="Skjul oplysningerne fra Den Danske Begrebsordbog">
<span class="ddb-close" onclick="$('#ddb_long_betydning-2').hide();$('#ddb_short_betydning-2').show()" role="button">...vis mindre</span>
</a>
<div>
<span class="tipIkon">
<img alt="tip" height="18" src="tip_ikon_mini.gif" width="18"/>
</span>
<a href="https://ordnet.dk/ddb">Læs mere om Den Danske Begrebsordbog</a>
</div>
</span>
</div>
</div>
<div class="definitionBox grammatik details">
<span class="stempel">grammatik</span>
<span class="inlineList allow-glossing">især med foranstillet adjektiv</span>
</div>
<div class="definitionBox">
<div class="citat-box"><span class="citat">Den dovne hund fik smag for det uvirksomme hængekøjeliv</span> <a class="kildepop" href="javascript:void(0);"><span class="kilde">BerlT1990</span><span class="popup">Berlingske Tidende (avis), 1990.</span></a></div>
</div>
<div class="definitionBox">
<div class="citat-box"><span class="citat">Skal jeg prøve at indhente ham? – Nej! Lad kun den feje hund løbe!</span> <a class="kildepop" href="javascript:void(0);"><span class="kilde">FamJour1985</span><span class="popup">Familie Journalen (blad), 1985.</span></a></div>
</div>
<div class="strokeGrey"> </div>
</div>
<div class="definitionNumber">3.
        </div>
<div class="definitionIndent">
<div class="definitionBox" id="betydning-3">
<span class="tekstmedium"><span class="stempelNoBorder">jargon</span> <span class="definition">solid firkantet plade der er påmonteret hjul, og som bl.a. bruges til at flytte tunge ting på</span></span>
</div>
<div class="definitionBox rel-begreber" id="plus-ddb-betydning-3">
<span id="ddb_short_betydning-3">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="performddbiddoQuery('21033606', 'ddb_long_betydning-3_insert', 'ddb_short_betydning-3', 'ddb_long_betydning-3')">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span class="level2 allow-glossing">
<a href="?entry_id=11041783&amp;def_id=91005319&amp;query=hund">påhængsvogn</a>
</span>
<a href="javascript:void(0)" title="Se flere oplysninger fra Den Danske Begrebsordbog">
<span class="ddb-open" id="ddb_open_betydning-3" onclick="performddbiddoQuery('21033606', 'ddb_long_betydning-3_insert', 'ddb_short_betydning-3', 'ddb_long_betydning-3')" role="button">...vis mere</span>
</a>
</span>
</span>
<div class="ddb-details" id="ddb_long_betydning-3" style="display:none;">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="$('#ddb_long_betydning-3').hide();$('#ddb_short_betydning-3').show();">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span id="ddb_long_betydning-3_insert"> </span>
<a href="javascript:void(0)" title="Skjul oplysningerne fra Den Danske Begrebsordbog">
<span class="ddb-close" onclick="$('#ddb_long_betydning-3').hide();$('#ddb_short_betydning-3').show()" role="button">...vis mindre</span>
</a>
<div>
<span class="tipIkon">
<img alt="tip" height="18" src="tip_ikon_mini.gif" width="18"/>
</span>
<a href="https://ordnet.dk/ddb">Læs mere om Den Danske Begrebsordbog</a>
</div>
</span>
</div>
</div>
</div>
</div>
<div class="openClose2">
<img alt="Skjul Faste udtryk" height="16" id="minus-faste-udtryk" onclick="lunchboxClose('faste-udtryk');" src="minus.gif" width="16"/>
<img alt="Vis Betydninger" height="16" id="plus-faste-udtryk" onclick="lunchboxOpen('faste-udtryk');" src="plus.gif" style="display: none;" width="16"/>
</div>
<div class="definitionBox">
<h2 id="faste-udtryk">Faste udtryk</h2>
</div>
<div id="content-faste-udtryk">
<div class="definitionBox" id="udtryk-1">
<span class="tekstlarge">
<span class="heavy underopslag">
<span class="kIkon details">
<a href="https://ordnet.dk/korpusdk/qconc?query=dér ligger hunden begravet" title='Slå "dér ligger hunden begravet" op i KorpusDK'>
<img alt="ikon for korpussøgning" height="11" src="k-ikon.gif" width="11"/>
</a>
</span>
<span class="match">dér ligger hunden begravet</span>
</span>
</span>
</div>
<div class="definitionNumber"> </div>
<div class="definitionIndent">
<div class="definitionBox" id="udtryk-1-betydning-1">
<span class="tekstmedium">
<span class="definition">dér er det egentlige problem</span>
</span>
</div>
<div class="definitionBox rel-begreber" id="plus-ddb-udtryk-1-betydning-1">
<span id="ddb_short_udtryk-1-betydning-1">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="performddbiddoQuery('21033608', 'ddb_long_udtryk-1-betydning-1_insert', 'ddb_short_udtryk-1-betydning-1', 'ddb_long_udtryk-1-betydning-1')">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList"><span class="level1 allow-glossing"><a href="?subentry_id=59009254&amp;def_id=21074578&amp;query=hund">skyldes</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?subentry_id=59005872&amp;def_id=21047728&amp;query=hund">der ligger noget bag</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?subentry_id=59010091&amp;def_id=21079853&amp;query=hund">der stikker noget under</a></span><a href="javascript:void(0)" title="Se flere oplysninger fra Den Danske Begrebsordbog"><span class="ddb-open" id="ddb_open_udtryk-1-betydning-1" onclick="performddbiddoQuery('21033608', 'ddb_long_udtryk-1-betydning-1_insert', 'ddb_short_udtryk-1-betydning-1', 'ddb_long_udtryk-1-betydning-1')" role="button">...vis mere</span></a></span>
</span>
<div class="ddb-details" id="ddb_long_udtryk-1-betydning-1" style="display:none;">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="$('#ddb_long_udtryk-1-betydning-1').hide();$('#ddb_short_udtryk-1-betydning-1').show();">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span id="ddb_long_udtryk-1-betydning-1_insert"> </span>
<a href="javascript:void(0)" title="Skjul oplysningerne fra Den Danske Begrebsordbog">
<span class="ddb-close" onclick="$('#ddb_long_udtryk-1-betydning-1').hide();$('#ddb_short_udtryk-1-betydning-1').show()" role="button">...vis mindre</span>
</a>
<div>
<span class="tipIkon">
<img alt="tip" height="18" src="tip_ikon_mini.gif" width="18"/>
</span>
<a href="https://ordnet.dk/ddb">Læs mere om Den Danske Begrebsordbog</a>
</div>
</span>
</div>
</div>
<div class="definitionBox">
<div class="citat-box"><span class="citat">Det drejer sig med andre ord om at forvente noget af de unge, siger han. Det er her, hunden ligger begravet</span> <a class="kildepop" href="javascript:void(0);"><span class="kilde">Morgenp1985</span><span class="popup">Nielsen, Mogens Frohn: En lykkelig mand der har slidt for sin; Morgenposten (Fyens Stiftstidende, søndag), 1985.</span></a></div>
</div>
<div class="strokeGrey"> </div>
</div>
<div class="definitionBox" id="udtryk-2">
<span class="tekstlarge">
<span class="heavy underopslag">
<span class="kIkon details">
<a href="https://ordnet.dk/korpusdk/qconc?query=det er mig, sagde hunden" title='Slå "det er mig, sagde hunden" op i KorpusDK'>
<img alt="ikon for korpussøgning" height="11" src="k-ikon.gif" width="11"/>
</a>
</span>
<span class="match">det er mig, sagde hunden</span>
</span>
</span>
</div>
<div class="definitionNumber"> </div>
<div class="definitionIndent">
<div class="definitionBox" id="udtryk-2-betydning-1">
<span class="tekstmedium">
<span class="stempelNoBorder">talemåde</span>
<span class="definition">bruges som bagatelliserende udtryk når man nævner sig selv i en sammenhæng hvor også andre indgår</span>
</span>
<span class="stempel">SPROGBRUG</span>
<span class="tekstnormal">spøgende</span>
</div>
<div class="definitionBox">
<div class="citat-box"><span class="kommentar">varierer i form</span> <span class="citat">der var kun mig sagde hunden, og min lillebror og min lillesøster</span> <a class="kildepop" href="javascript:void(0);"><span class="kilde">talesp1993</span><span class="popup">talesprog, int, 1993</span></a></div>
</div>
<div class="strokeGrey"> </div>
</div>
<div class="definitionBox" id="udtryk-3">
<span class="tekstlarge">
<span class="heavy underopslag">
<span class="kIkon details">
<a href="https://ordnet.dk/korpusdk/qconc?query=en hund i et spil kegler" title='Slå "en hund i et spil kegler" op i KorpusDK'>
<img alt="ikon for korpussøgning" height="11" src="k-ikon.gif" width="11"/>
</a>
</span>
<span class="match">en hund i et spil kegler</span>
</span>
</span>
</div>
<div class="definitionNumber"> </div>
<div class="definitionIndent">
<div class="definitionBox" id="udtryk-3-betydning-1">
<span class="tekstmedium">
<span class="stempelNoBorder">overført</span>
<span class="definition">person, genstand el.lign. der virker fremmed eller malplaceret i en bestemt sammenhæng</span>
</span>
</div>
<div class="definitionBox rel-begreber" id="plus-ddb-udtryk-3-betydning-1">
<span id="ddb_short_udtryk-3-betydning-1">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="performddbiddoQuery('21033610', 'ddb_long_udtryk-3-betydning-1_insert', 'ddb_short_udtryk-3-betydning-1', 'ddb_long_udtryk-3-betydning-1')">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList"><span class="level1 allow-glossing"><a href="?entry_id=12003969&amp;def_id=21090717&amp;query=hund">udlænding</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11038135&amp;def_id=21059952&amp;query=hund">outsider</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?subentry_id=59002767&amp;def_id=21024936&amp;query=hund">fremmed fugl</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?subentry_id=59009896&amp;def_id=21078612&amp;query=hund">en spurv i tranedans</a></span><a href="javascript:void(0)" title="Se flere oplysninger fra Den Danske Begrebsordbog"><span class="ddb-open" id="ddb_open_udtryk-3-betydning-1" onclick="performddbiddoQuery('21033610', 'ddb_long_udtryk-3-betydning-1_insert', 'ddb_short_udtryk-3-betydning-1', 'ddb_long_udtryk-3-betydning-1')" role="button">...vis mere</span></a></span>
</span>
<div class="ddb-details" id="ddb_long_udtryk-3-betydning-1" style="display:none;">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="$('#ddb_long_udtryk-3-betydning-1').hide();$('#ddb_short_udtryk-3-betydning-1').show();">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span id="ddb_long_udtryk-3-betydning-1_insert"> </span>
<a href="javascript:void(0)" title="Skjul oplysningerne fra Den Danske Begrebsordbog">
<span class="ddb-close" onclick="$('#ddb_long_udtryk-3-betydning-1').hide();$('#ddb_short_udtryk-3-betydning-1').show()" role="button">...vis mindre</span>
</a>
<div>
<span class="tipIkon">
<img alt="tip" height="18" src="tip_ikon_mini.gif" width="18"/>
</span>
<a href="https://ordnet.dk/ddb">Læs mere om Den Danske Begrebsordbog</a>
</div>
</span>
</div>
</div>
<div class="definitionBox">
<div class="citat-box"><span class="citat">Halogenlamper ville være en hund i et spil kegler i et rum som dette</span> <a class="kildepop" href="javascript:void(0);"><span class="kilde">BoBedre1992</span><span class="popup">BO BEDRE (blad), 1992.</span></a></div>
</div>
<div class="strokeGrey"> </div>
</div>
<div class="definitionBox" id="udtryk-4">
<span class="tekstlarge">
<span class="heavy underopslag">
<span class="kIkon details">
<a href="https://ordnet.dk/korpusdk/qconc?query=flyvende hund" title='Slå "flyvende hund" op i KorpusDK'>
<img alt="ikon for korpussøgning" height="11" src="k-ikon.gif" width="11"/>
</a>
</span>
<span class="match">flyvende hund</span>
</span>
</span>
</div>
<div class="definitionNumber"> </div>
<div class="definitionIndent">
<div class="definitionBox" id="udtryk-4-betydning-1">
<span class="tekstmedium"><span class="stempelNoBorder">ZOOLOGI</span> <span class="definition">stor flagermus som er udbredt i Afrika, Asien og Australien og bl.a. lever af frugt; visse arter har et hundelignende hoved</span></span>
</div>
<div class="definitionBox onym">
<span class="stempel">Synonym</span>
<span class="inlineList"><span class="diskret">familien</span> Pteropidae</span>
</div>
<div class="definitionBox rel-begreber" id="plus-ddb-udtryk-4-betydning-1">
<span id="ddb_short_udtryk-4-betydning-1">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="performddbiddoQuery('21033611', 'ddb_long_udtryk-4-betydning-1_insert', 'ddb_short_udtryk-4-betydning-1', 'ddb_long_udtryk-4-betydning-1')">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList"><span class="level3 allow-glossing"><a href="?entry_id=11050512&amp;def_id=21080407&amp;query=hund">storflagermus</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11048154&amp;def_id=21076288&amp;query=hund">småflagermus</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=12005621&amp;def_id=21093355&amp;query=hund">vampyr</a></span><a href="javascript:void(0)" title="Se flere oplysninger fra Den Danske Begrebsordbog"><span class="ddb-open" id="ddb_open_udtryk-4-betydning-1" onclick="performddbiddoQuery('21033611', 'ddb_long_udtryk-4-betydning-1_insert', 'ddb_short_udtryk-4-betydning-1', 'ddb_long_udtryk-4-betydning-1')" role="button">...vis mere</span></a></span>
</span>
<div class="ddb-details" id="ddb_long_udtryk-4-betydning-1" style="display:none;">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="$('#ddb_long_udtryk-4-betydning-1').hide();$('#ddb_short_udtryk-4-betydning-1').show();">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span id="ddb_long_udtryk-4-betydning-1_insert"> </span>
<a href="javascript:void(0)" title="Skjul oplysningerne fra Den Danske Begrebsordbog">
<span class="ddb-close" onclick="$('#ddb_long_udtryk-4-betydning-1').hide();$('#ddb_short_udtryk-4-betydning-1').show()" role="button">...vis mindre</span>
</a>
<div>
<span class="tipIkon">
<img alt="tip" height="18" src="tip_ikon_mini.gif" width="18"/>
</span>
<a href="https://ordnet.dk/ddb">Læs mere om Den Danske Begrebsordbog</a>
</div>
</span>
</div>
</div>
<div class="definitionBox">
<div class="citat-box"><span class="citat">Tusinder af flyvende hunde tegnede en mørk stribe på himlen</span> <a class="kildepop" href="javascript:void(0);"><span class="kilde">ViJung89</span><span class="popup">Vilhelm Jung: Det var det. Vilhelm Jung fortæller vide. Gyldendal, 1989.</span></a></div>
</div>
<div class="strokeGrey"> </div>
</div>
<div class="definitionBox" id="udtryk-5">
<span class="tekstlarge">
<span class="heavy underopslag">
<span class="kIkon details">
<a href="https://ordnet.dk/korpusdk/qconc?query=gå i hundene" title='Slå "gå i hundene" op i KorpusDK'>
<img alt="ikon for korpussøgning" height="11" src="k-ikon.gif" width="11"/>
</a>
</span>
<span class="match">gå i hundene</span>
</span>
</span>
</div>
<div class="definitionNumber"> </div>
<div class="definitionIndent">
<div class="definitionBox" id="udtryk-5-betydning-1">
<span class="tekstmedium">
<span class="stempelNoBorder">overført</span>
<span class="definition">gå til grunde økonomisk, socialt og fysisk, ofte i forbindelse med alkoholmisbrug</span>
</span>
</div>
<div class="definitionBox rel-begreber" id="plus-ddb-udtryk-5-betydning-1">
<span id="ddb_short_udtryk-5-betydning-1">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="performddbiddoQuery('21033612', 'ddb_long_udtryk-5-betydning-1_insert', 'ddb_short_udtryk-5-betydning-1', 'ddb_long_udtryk-5-betydning-1')">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList"><span class="level3 allow-glossing"><a href="?subentry_id=59003556&amp;def_id=21030006&amp;query=hund">gå nedenom og hjem</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?subentry_id=59003335&amp;def_id=21028752&amp;query=hund">gå til grunde</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11020023&amp;def_id=21030921&amp;query=hund">havarere</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11046465&amp;def_id=21073086&amp;query=hund">lide skibbrud</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?subentry_id=59007956&amp;def_id=21065674&amp;query=hund">gå rabundus</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?subentry_id=59006554&amp;def_id=21052264&amp;query=hund">aldrig blive menneske igen</a></span><a href="javascript:void(0)" title="Se flere oplysninger fra Den Danske Begrebsordbog"><span class="ddb-open" id="ddb_open_udtryk-5-betydning-1" onclick="performddbiddoQuery('21033612', 'ddb_long_udtryk-5-betydning-1_insert', 'ddb_short_udtryk-5-betydning-1', 'ddb_long_udtryk-5-betydning-1')" role="button">...vis mere</span></a></span>
</span>
<div class="ddb-details" id="ddb_long_udtryk-5-betydning-1" style="display:none;">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="$('#ddb_long_udtryk-5-betydning-1').hide();$('#ddb_short_udtryk-5-betydning-1').show();">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span id="ddb_long_udtryk-5-betydning-1_insert"> </span>
<a href="javascript:void(0)" title="Skjul oplysningerne fra Den Danske Begrebsordbog">
<span class="ddb-close" onclick="$('#ddb_long_udtryk-5-betydning-1').hide();$('#ddb_short_udtryk-5-betydning-1').show()" role="button">...vis mindre</span>
</a>
<div>
<span class="tipIkon">
<img alt="tip" height="18" src="tip_ikon_mini.gif" width="18"/>
</span>
<a href="https://ordnet.dk/ddb">Læs mere om Den Danske Begrebsordbog</a>
</div>
</span>
</div>
</div>
<div class="definitionBox">
<div class="citat-box"><span class="citat">Derfor saboterede jeg mit liv, og gjorde alt for at gå i hundene. Jeg prøvede endda at skabe et alkohol-problem</span> <a class="kildepop" href="javascript:void(0);"><span class="kilde">BT1991</span><span class="popup">B.T. (avis), 1991.</span></a></div>
</div>
<div class="strokeGrey"> </div>
</div>
<div class="definitionBox" id="udtryk-6">
<span class="tekstlarge">
<span class="heavy underopslag">
<span class="kIkon details">
<a href="https://ordnet.dk/korpusdk/qconc?query=hård hund" title='Slå "hård hund" op i KorpusDK'>
<img alt="ikon for korpussøgning" height="11" src="k-ikon.gif" width="11"/>
</a>
</span>
<span class="match">hård hund</span>
</span>
</span>
</div>
<div class="definitionNumber"> </div>
<div class="definitionIndent">
<div class="definitionBox" id="udtryk-6-betydning-1">
<span class="tekstmedium">
<span class="stempelNoBorder">overført</span>
<span class="definition">ufølsom person</span>
</span>
<span class="stempel">SPROGBRUG</span>
<span class="tekstnormal">uformelt</span>
</div>
<div class="definitionBox onym">
<span class="stempel">Synonym</span>
<span class="inlineList">
<a href="?subentry_id=59006947&amp;def_id=21056068&amp;query=hund">hård negl</a>
</span>
</div>
<div class="definitionBox rel-begreber" id="plus-ddb-udtryk-6-betydning-1">
<span id="ddb_short_udtryk-6-betydning-1">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="performddbiddoQuery('21033613', 'ddb_long_udtryk-6-betydning-1_insert', 'ddb_short_udtryk-6-betydning-1', 'ddb_long_udtryk-6-betydning-1')">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList"><span class="level3 allow-glossing"><a href="?entry_id=11022302&amp;def_id=21034733&amp;query=hund">høg</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11028979&amp;def_id=21045341&amp;query=hund">kyniker</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?subentry_id=59000441&amp;def_id=21004932&amp;query=hund">hård banan</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?subentry_id=59007066&amp;def_id=21057030&amp;query=hund">hård nyser</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?subentry_id=59003625&amp;def_id=21030299&amp;query=hund">hård hals</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=45001993&amp;def_id=28115495&amp;query=hund">hardhitter</a></span><a href="javascript:void(0)" title="Se flere oplysninger fra Den Danske Begrebsordbog"><span class="ddb-open" id="ddb_open_udtryk-6-betydning-1" onclick="performddbiddoQuery('21033613', 'ddb_long_udtryk-6-betydning-1_insert', 'ddb_short_udtryk-6-betydning-1', 'ddb_long_udtryk-6-betydning-1')" role="button">...vis mere</span></a></span>
</span>
<div class="ddb-details" id="ddb_long_udtryk-6-betydning-1" style="display:none;">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="$('#ddb_long_udtryk-6-betydning-1').hide();$('#ddb_short_udtryk-6-betydning-1').show();">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span id="ddb_long_udtryk-6-betydning-1_insert"> </span>
<a href="javascript:void(0)" title="Skjul oplysningerne fra Den Danske Begrebsordbog">
<span class="ddb-close" onclick="$('#ddb_long_udtryk-6-betydning-1').hide();$('#ddb_short_udtryk-6-betydning-1').show()" role="button">...vis mindre</span>
</a>
<div>
<span class="tipIkon">
<img alt="tip" height="18" src="tip_ikon_mini.gif" width="18"/>
</span>
<a href="https://ordnet.dk/ddb">Læs mere om Den Danske Begrebsordbog</a>
</div>
</span>
</div>
</div>
<div class="definitionBox">
<div class="citat-box"><span class="citat">Hun var en hård hund, der nådeløst slæbte mig til papirbunkerne, når de blev for høje</span> <a class="kildepop" href="javascript:void(0);"><span class="kilde">HaSølv89</span><span class="popup">Hans Sølvhøj: Rødt på hvidt. Gyldendal, 1989.</span></a></div>
</div>
<div class="strokeGrey"> </div>
</div>
<div class="definitionBox" id="udtryk-7">
<span class="tekstlarge">
<span class="heavy underopslag">
<span class="kIkon details">
<a href="https://ordnet.dk/korpusdk/qconc?query=leve som hund og kat" title='Slå "leve som hund og kat" op i KorpusDK'>
<img alt="ikon for korpussøgning" height="11" src="k-ikon.gif" width="11"/>
</a>
</span>
<span class="match">leve som hund og kat</span>
</span>
</span>
</div>
<div class="definitionNumber"> </div>
<div class="definitionIndent">
<div class="definitionBox" id="udtryk-7-betydning-1">
<span class="tekstmedium">
<span class="definition">leve sammen og altid være uenige og skændes</span>
</span>
</div>
<div class="definitionBox rel-begreber" id="plus-ddb-udtryk-7-betydning-1">
<span id="ddb_short_udtryk-7-betydning-1">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="performddbiddoQuery('21033614', 'ddb_long_udtryk-7-betydning-1_insert', 'ddb_short_udtryk-7-betydning-1', 'ddb_long_udtryk-7-betydning-1')">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span class="level2 allow-glossing">
<a href="?subentry_id=59009285&amp;def_id=21074726&amp;query=hund">skændes</a>
</span>
<a href="javascript:void(0)" title="Se flere oplysninger fra Den Danske Begrebsordbog">
<span class="ddb-open" id="ddb_open_udtryk-7-betydning-1" onclick="performddbiddoQuery('21033614', 'ddb_long_udtryk-7-betydning-1_insert', 'ddb_short_udtryk-7-betydning-1', 'ddb_long_udtryk-7-betydning-1')" role="button">...vis mere</span>
</a>
</span>
</span>
<div class="ddb-details" id="ddb_long_udtryk-7-betydning-1" style="display:none;">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="$('#ddb_long_udtryk-7-betydning-1').hide();$('#ddb_short_udtryk-7-betydning-1').show();">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span id="ddb_long_udtryk-7-betydning-1_insert"> </span>
<a href="javascript:void(0)" title="Skjul oplysningerne fra Den Danske Begrebsordbog">
<span class="ddb-close" onclick="$('#ddb_long_udtryk-7-betydning-1').hide();$('#ddb_short_udtryk-7-betydning-1').show()" role="button">...vis mindre</span>
</a>
<div>
<span class="tipIkon">
<img alt="tip" height="18" src="tip_ikon_mini.gif" width="18"/>
</span>
<a href="https://ordnet.dk/ddb">Læs mere om Den Danske Begrebsordbog</a>
</div>
</span>
</div>
</div>
<div class="definitionBox">
<div class="citat-box"><span class="citat">Vi kender alle sammen par, der lever sammen som hund og kat – og hvor man undrer sig over, at de ikke bliver skilt</span> <a class="kildepop" href="javascript:void(0);"><span class="kilde">IForm1992</span><span class="popup">I FORM (blad), 1992.</span></a></div>
</div>
<div class="strokeGrey"> </div>
</div>
<div class="definitionBox" id="udtryk-8">
<span class="tekstlarge">
<span class="heavy underopslag">
<span class="kIkon details">
<a href="https://ordnet.dk/korpusdk/qconc?query=man kan ikke lære en gammel hund nye kunster" title='Slå "man kan ikke lære en gammel hund nye kunster" op i KorpusDK'>
<img alt="ikon for korpussøgning" height="11" src="k-ikon.gif" width="11"/>
</a>
</span>
<span class="match">man kan ikke lære en gammel hund nye kunster</span>
</span>
</span>
</div>
<div class="definitionNumber"> </div>
<div class="definitionIndent">
<div class="definitionBox" id="udtryk-8-betydning-1">
<span class="tekstmedium">
<span class="stempelNoBorder">talemåde</span>
<span class="definition">en (ældre) person med indgroede vaner har svært ved at omstille sig og lære nyt</span>
</span>
</div>
<div class="definitionBox rel-begreber" id="plus-ddb-udtryk-8-betydning-1">
<span id="ddb_short_udtryk-8-betydning-1">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="performddbiddoQuery('21033615', 'ddb_long_udtryk-8-betydning-1_insert', 'ddb_short_udtryk-8-betydning-1', 'ddb_long_udtryk-8-betydning-1')">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span class="level3 allow-glossing">
<a href="?subentry_id=59003425&amp;def_id=21029290&amp;query=hund">spildte Guds ord på Balle-Lars</a>
</span>
<a href="javascript:void(0)" title="Se flere oplysninger fra Den Danske Begrebsordbog">
<span class="ddb-open" id="ddb_open_udtryk-8-betydning-1" onclick="performddbiddoQuery('21033615', 'ddb_long_udtryk-8-betydning-1_insert', 'ddb_short_udtryk-8-betydning-1', 'ddb_long_udtryk-8-betydning-1')" role="button">...vis mere</span>
</a>
</span>
</span>
<div class="ddb-details" id="ddb_long_udtryk-8-betydning-1" style="display:none;">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="$('#ddb_long_udtryk-8-betydning-1').hide();$('#ddb_short_udtryk-8-betydning-1').show();">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span id="ddb_long_udtryk-8-betydning-1_insert"> </span>
<a href="javascript:void(0)" title="Skjul oplysningerne fra Den Danske Begrebsordbog">
<span class="ddb-close" onclick="$('#ddb_long_udtryk-8-betydning-1').hide();$('#ddb_short_udtryk-8-betydning-1').show()" role="button">...vis mindre</span>
</a>
<div>
<span class="tipIkon">
<img alt="tip" height="18" src="tip_ikon_mini.gif" width="18"/>
</span>
<a href="https://ordnet.dk/ddb">Læs mere om Den Danske Begrebsordbog</a>
</div>
</span>
</div>
</div>
<div class="definitionBox">
<div class="citat-box"><span class="citat">Hvis jeg havde været yngre, havde jeg tilpasset mig, men man kan ikke lære en gammel hund nye kunster</span> <a class="kildepop" href="javascript:void(0);"><span class="kilde">BerlT1991</span><span class="popup">Berlingske Tidende (avis), 1991.</span></a></div>
</div>
<div class="strokeGrey"> </div>
</div>
<div class="definitionBox" id="udtryk-9">
<span class="tekstlarge">
<span class="heavy underopslag">
<span class="kIkon details">
<a href="https://ordnet.dk/korpusdk/qconc?query=man skal ikke skue hunden på hårene" title='Slå "man skal ikke skue hunden på hårene" op i KorpusDK'>
<img alt="ikon for korpussøgning" height="11" src="k-ikon.gif" width="11"/>
</a>
</span>
<span class="match">man skal ikke skue hunden på hårene</span>
</span>
</span>
</div>
<div class="definitionNumber"> </div>
<div class="definitionIndent">
<div class="definitionBox" id="udtryk-9-betydning-1">
<span class="tekstmedium">
<span class="stempelNoBorder">talemåde</span>
<span class="definition">man skal ikke bedømme noget alene ud fra dets ydre</span>
</span>
</div>
<div class="definitionBox onym">
<span class="stempel">Se også</span>
<span class="inlineList">
<a href="?subentry_id=59009045&amp;def_id=21073320&amp;query=hund">skinnet bedrager</a>
</span>
</div>
<div class="definitionBox rel-begreber" id="plus-ddb-udtryk-9-betydning-1">
<span id="ddb_short_udtryk-9-betydning-1">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="performddbiddoQuery('21033616', 'ddb_long_udtryk-9-betydning-1_insert', 'ddb_short_udtryk-9-betydning-1', 'ddb_long_udtryk-9-betydning-1')">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList"><span class="level3 allow-glossing"><a href="?subentry_id=59009045&amp;def_id=21073320&amp;query=hund">skinnet bedrager</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?subentry_id=59008666&amp;def_id=21070878&amp;query=hund">man ser det for sig</a></span><a href="javascript:void(0)" title="Se flere oplysninger fra Den Danske Begrebsordbog"><span class="ddb-open" id="ddb_open_udtryk-9-betydning-1" onclick="performddbiddoQuery('21033616', 'ddb_long_udtryk-9-betydning-1_insert', 'ddb_short_udtryk-9-betydning-1', 'ddb_long_udtryk-9-betydning-1')" role="button">...vis mere</span></a></span>
</span>
<div class="ddb-details" id="ddb_long_udtryk-9-betydning-1" style="display:none;">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="$('#ddb_long_udtryk-9-betydning-1').hide();$('#ddb_short_udtryk-9-betydning-1').show();">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span id="ddb_long_udtryk-9-betydning-1_insert"> </span>
<a href="javascript:void(0)" title="Skjul oplysningerne fra Den Danske Begrebsordbog">
<span class="ddb-close" onclick="$('#ddb_long_udtryk-9-betydning-1').hide();$('#ddb_short_udtryk-9-betydning-1').show()" role="button">...vis mindre</span>
</a>
<div>
<span class="tipIkon">
<img alt="tip" height="18" src="tip_ikon_mini.gif" width="18"/>
</span>
<a href="https://ordnet.dk/ddb">Læs mere om Den Danske Begrebsordbog</a>
</div>
</span>
</div>
</div>
<div class="definitionBox">
<div class="citat-box"><span class="citat">man [skulle ikke sådan] skue hunden på hårene – der kan jo udmærket være en lækker lejlighed i et forfaldent hus</span> <a class="kildepop" href="javascript:void(0);"><span class="kilde">privbrev85</span><span class="popup">privatbrev, 1985.</span></a></div>
</div>
<div class="strokeGrey"> </div>
</div>
<div class="definitionBox" id="udtryk-10">
<span class="tekstlarge">
<span class="heavy underopslag">
<span class="kIkon details">
<a href="https://ordnet.dk/korpusdk/qconc?query=røde hunde" title='Slå "røde hunde" op i KorpusDK'>
<img alt="ikon for korpussøgning" height="11" src="k-ikon.gif" width="11"/>
</a>
</span>
<span class="match">røde hunde</span>
</span>
</span>
</div>
<div class="definitionNumber"> </div>
<div class="definitionIndent">
<div class="definitionBox" id="udtryk-10-betydning-1">
<span class="tekstmedium"><span class="definition">børnesygdom forårsaget af en virusinfektion, med let feber, ondt i halsen, hævede lymfeknuder og rødt udslæt</span><span class="dividerDot"> </span>kan give fosterskader hvis den rammer gravide tidligt i graviditeten</span>
</div>
<div class="definitionBox rel-begreber" id="plus-ddb-udtryk-10-betydning-1">
<span id="ddb_short_udtryk-10-betydning-1">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="performddbiddoQuery('21033617', 'ddb_long_udtryk-10-betydning-1_insert', 'ddb_short_udtryk-10-betydning-1', 'ddb_long_udtryk-10-betydning-1')">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList"><span class="level3 allow-glossing"><a href="?entry_id=11016978&amp;def_id=21026008&amp;query=hund">fåresyge</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11025869&amp;def_id=21040271&amp;query=hund">kighoste</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11031251&amp;def_id=21049101&amp;query=hund">lussingesyge</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11046331&amp;def_id=21072887&amp;query=hund">skarlagensfeber</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11046705&amp;def_id=21073496&amp;query=hund">skoldkopper</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11034966&amp;def_id=21054765&amp;query=hund">mæslinger</a></span><a href="javascript:void(0)" title="Se flere oplysninger fra Den Danske Begrebsordbog"><span class="ddb-open" id="ddb_open_udtryk-10-betydning-1" onclick="performddbiddoQuery('21033617', 'ddb_long_udtryk-10-betydning-1_insert', 'ddb_short_udtryk-10-betydning-1', 'ddb_long_udtryk-10-betydning-1')" role="button">...vis mere</span></a></span>
</span>
<div class="ddb-details" id="ddb_long_udtryk-10-betydning-1" style="display:none;">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="$('#ddb_long_udtryk-10-betydning-1').hide();$('#ddb_short_udtryk-10-betydning-1').show();">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span id="ddb_long_udtryk-10-betydning-1_insert"> </span>
<a href="javascript:void(0)" title="Skjul oplysningerne fra Den Danske Begrebsordbog">
<span class="ddb-close" onclick="$('#ddb_long_udtryk-10-betydning-1').hide();$('#ddb_short_udtryk-10-betydning-1').show()" role="button">...vis mindre</span>
</a>
<div>
<span class="tipIkon">
<img alt="tip" height="18" src="tip_ikon_mini.gif" width="18"/>
</span>
<a href="https://ordnet.dk/ddb">Læs mere om Den Danske Begrebsordbog</a>
</div>
</span>
</div>
</div>
<div class="definitionBox">
<div class="citat-box"><span class="citat">Blodprøven viste, at denne unge kvinde ikke tidligere havde haft røde hunde og hun bør derfor vaccineres</span> <a class="kildepop" href="javascript:void(0);"><span class="kilde">Helse1986</span><span class="popup">Helse (blad), 1986.</span></a></div>
</div>
<div class="strokeGrey"> </div>
</div>
<div class="definitionBox" id="udtryk-11">
<span class="tekstlarge">
<span class="heavy underopslag">
<span class="kIkon details">
<a href="https://ordnet.dk/korpusdk/qconc?query=vittig hund" title='Slå "vittig hund" op i KorpusDK'>
<img alt="ikon for korpussøgning" height="11" src="k-ikon.gif" width="11"/>
</a>
</span>
<span class="match">vittig hund</span>
</span>
</span>
</div>
<div class="definitionNumber"> </div>
<div class="definitionIndent">
<div class="definitionBox" id="udtryk-11-betydning-1">
<span class="tekstmedium">
<span class="stempelNoBorder">overført</span>
<span class="definition">vittig person</span>
</span>
<span class="stempel">SPROGBRUG</span>
<span class="tekstnormal">uformelt</span>
</div>
<div class="definitionBox onym">
<span class="stempel">Synonym</span>
<span class="inlineList">vittigt hoved</span>
</div>
<div class="definitionBox">
<div class="citat-box"><span class="citat">Hvorfor se Neapel og dø, når man kan kede sig ihjel i Hadsten. Det har en vittig hund engang foreslået som slogan for byen</span> <a class="kildepop" href="javascript:void(0);"><span class="kilde">HadstFbl1992</span><span class="popup">Hadsten og Omegns Folkeblad (lokalavis), 1992.</span></a></div>
</div>
<div class="strokeGrey"> </div>
</div>
<div class="definitionBox" id="udtryk-12">
<span class="tekstlarge">
<span class="heavy underopslag">
<span class="kIkon details">
<a href="https://ordnet.dk/korpusdk/qconc?query=være en hund efter noget" title='Slå "være en hund efter noget" op i KorpusDK'>
<img alt="ikon for korpussøgning" height="11" src="k-ikon.gif" width="11"/>
</a>
</span>
<span class="match">være en hund efter noget</span>
</span>
</span>
</div>
<div class="definitionNumber"> </div>
<div class="definitionIndent">
<div class="definitionBox" id="udtryk-12-betydning-1">
<span class="tekstmedium">
<span class="definition">have stor interesse for eller lyst til noget</span>
</span>
</div>
<div class="definitionBox rel-begreber" id="plus-ddb-udtryk-12-betydning-1">
<span id="ddb_short_udtryk-12-betydning-1">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="performddbiddoQuery('21033619', 'ddb_long_udtryk-12-betydning-1_insert', 'ddb_short_udtryk-12-betydning-1', 'ddb_long_udtryk-12-betydning-1')">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList"><span class="level3 allow-glossing"><a href="?entry_id=11021691&amp;def_id=21033692&amp;query=hund">hungre efter</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11051383&amp;def_id=21082021&amp;query=hund">sulte efter</a></span> <span class="dividerSmall"> </span> <span class="level3 allow-glossing"><a href="?entry_id=11051343&amp;def_id=21081965&amp;query=hund">sukke efter</a></span><a href="javascript:void(0)" title="Se flere oplysninger fra Den Danske Begrebsordbog"><span class="ddb-open" id="ddb_open_udtryk-12-betydning-1" onclick="performddbiddoQuery('21033619', 'ddb_long_udtryk-12-betydning-1_insert', 'ddb_short_udtryk-12-betydning-1', 'ddb_long_udtryk-12-betydning-1')" role="button">...vis mere</span></a></span>
</span>
<div class="ddb-details" id="ddb_long_udtryk-12-betydning-1" style="display:none;">
<span alt="De nærmeste begreber fra Den Danske Begrebsordbog" class="short-tooltip" onclick="$('#ddb_long_udtryk-12-betydning-1').hide();$('#ddb_short_udtryk-12-betydning-1').show();">
<span class="stempel" title="">Ord i nærheden</span>
</span>
<span class="inlineList">
<span id="ddb_long_udtryk-12-betydning-1_insert"> </span>
<a href="javascript:void(0)" title="Skjul oplysningerne fra Den Danske Begrebsordbog">
<span class="ddb-close" onclick="$('#ddb_long_udtryk-12-betydning-1').hide();$('#ddb_short_udtryk-12-betydning-1').show()" role="button">...vis mindre</span>
</a>
<div>
<span class="tipIkon">
<img alt="tip" height="18" src="tip_ikon_mini.gif" width="18"/>
</span>
<a href="https://ordnet.dk/ddb">Læs mere om Den Danske Begrebsordbog</a>
</div>
</span>
</div>
</div>
<div class="definitionBox">
<div class="citat-box"><span class="citat">Jeg er en hund efter enhver form for quiz og selskabslege</span> <a class="kildepop" href="javascript:void(0);"><span class="kilde">UdeHjem1985</span><span class="popup">Ude og Hjemme (blad), 1985.</span></a></div>
</div>
</div>
</div>
<div class="openClose2">
<img alt="Skjul Betydninger" height="16" id="minus-orddannelser" onclick="lunchboxClose('orddannelser');" src="minus.gif" width="16"/>
<img alt="Vis Betydninger" height="16" id="plus-orddannelser" onclick="lunchboxOpen('orddannelser');" src="plus.gif" style="display: none;" width="16"/>
</div>
<div class="definitionBox">
<h2 id="orddannelser">Orddannelser</h2>
</div>
<div id="content-orddannelser">
<div class="definitionBox" id="id-afl">
<span class="stempel">Afledninger</span>
<span class="inlineList"><a href="?entry_id=11021687&amp;query=hund" title="til betydning 1">hundsk</a>adj.</span>
</div>
<div class="definitionBox" id="id-sms">
<span class="stempel">Sammensætninger</span>
<span class="inlineList"><a href="?entry_id=40001851&amp;query=hund" title="til betydning 1">hundebid</a> <span class="dividerSmall"> </span> <a href="?entry_id=50003709&amp;query=hund" title="til betydning 1">hundehale</a> <span class="dividerSmall"> </span> <a href="?entry_id=50003710&amp;query=hund" title="til betydning 1">hundehalsbånd</a> <span class="dividerSmall"> </span> <a href="?entry_id=40001852&amp;query=hund" title="til betydning 1">hundehår</a> <span class="dividerSmall"> </span> <a href="?entry_id=53001264&amp;query=hund" title="til betydning 1">hundekennel</a> <span class="dividerSmall"> </span> <a href="?entry_id=40001853&amp;query=hund" title="til betydning 1">hundekiks</a> <span class="dividerSmall"> </span> <a href="?entry_id=53001265&amp;query=hund" title="til betydning 1">hundekurv</a> <span class="dividerSmall"> </span> <a href="?entry_id=50003717&amp;query=hund" title="til betydning 1">hundelort</a> <span class="dividerSmall"> </span> <a href="?entry_id=53001266&amp;query=hund" title="til betydning 1">hundemad</a> <span class="dividerSmall"> </span> <a href="?entry_id=53001267&amp;query=hund" title="til betydning 1">hundeven</a> <span class="dividerDouble"> </span> <a href="?entry_id=11005179&amp;query=hund" title="til betydning 1">blodhund</a> <span class="dividerSmall"> </span> <a href="?entry_id=11012236&amp;query=hund" title="til betydning 1">familiehund</a> <span class="dividerSmall"> </span> <a href="?entry_id=11018561&amp;query=hund" title="til betydning 1">gravhund</a> <span class="dividerSmall"> </span> <a href="?entry_id=50003345&amp;query=hund" title="til betydning 1">hanhund</a> <span class="dividerSmall"> </span> <a href="?entry_id=50003731&amp;query=hund" title="til betydning 1">hunhund</a> <span class="dividerSmall"> </span> <a href="?entry_id=11024359&amp;query=hund" title="til betydning 1">jagthund</a> <span class="dividerSmall"> </span> <a href="?entry_id=45000946&amp;query=hund" title="til betydning 1">muskelhund</a> <span class="dividerSmall"> </span> <a href="?entry_id=50007627&amp;query=hund" title="til betydning 1">racehund</a> <span class="dividerSmall"> </span> <a href="?entry_id=11047864&amp;query=hund" title="til betydning 1">slædehund</a> <span class="dividerSmall"> </span> <a href="?entry_id=12005488&amp;query=hund" title="til betydning 1">vagthund</a></span>
</div>
</div>
<span class="artikelkilde"><a class="fejlrapport" href="mailto:ddo@dsl.dk?subject=Ang.%20Den%20Danske%20Ordbog:%20hund1%20(11021628)" target="_blank" title="Send os en mail hvis du opdager en fejl. Spørgsmål besvares desværre ikke.">Rapportér et problem</a>fra <a class="kildepop" href="javascript:void(0);"><span class="kilde">Den Danske Ordbog</span><span class="popup"><i>Den Danske Ordbog</i>. Det Danske Sprog- og Litteraturselskab 2023.
        Første udgave af ordbogen udkom 2003-5.<br/><br/>Denne artikel blev første gang udgivet i den trykte version af ordbogen <span class="year">2003-5</span> og senest opdateret <span class="year">2009</span>.
        </span></a></span>
</div>
"""

html_fixer(div)

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 20161 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}